In [21]:
import pandas as pd
import numpy as np
import env
import itertools as it
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
DB_URL = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/tidy_data'

1. Attendance Data

Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

You should end up with something like this:

name
Billy    0.5250

Jane     0.6875

John     0.9125

Sally    0.7625

Name: grade, dtype: float64


In [75]:
# create a dataframe from raw data above
df = pd.DataFrame({'name':['Billy', 'Jane', 'John', 'Sally']})
# create a dataframe with the grade column
df['attendence'] = [0.5250, 0.6875, 0.9125, 0.7625]
df


,name,attendence
0,Billy,0.5250
1,Jane,0.6875
2,John,0.9125
3,Sally,0.7625


In [78]:
# use attendence data
query = """
SELECT *
FROM attendance
""" 

df = pd.read_sql(query, DB_URL)
df.head()

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [79]:
# rename Unnamed: 0 to name
df = df.rename(columns={'Unnamed: 0':'name'})
df.head()

,name,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [80]:
# melt the dataframe
df = df.melt(id_vars=['name'], var_name='date', value_name='attendance') # id_vars is the column that will be used as the index. var_name is the column that will be used as the column name. value_name is the column that will be used as the value.
df

,name,date,attendance
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T
5,Jane,2018-01-02,P
6,Billy,2018-01-02,T
7,John,2018-01-02,T
8,Sally,2018-01-03,T
9,Jane,2018-01-03,T


In [81]:
# create a new column that holds the attendance value
df[['attendance_value']] = df['attendance'].str.split('%', expand=True)
# replace attendance_value varibles with 'H' = .5 and 'A' = 0  and P = 1 and 'T' = .9
df['attendance_value'] = df['attendance_value'].replace('H', .5)
df['attendance_value'] = df['attendance_value'].replace('A', 0)
df['attendance_value'] = df['attendance_value'].replace('P', 1)
df['attendance_value'] = df['attendance_value'].replace('T', .9)
df

,name,date,attendance,attendance_value
0,Sally,2018-01-01,P,1.0
1,Jane,2018-01-01,A,0.0
2,Billy,2018-01-01,A,0.0
3,John,2018-01-01,P,1.0
4,Sally,2018-01-02,T,0.9
5,Jane,2018-01-02,P,1.0
6,Billy,2018-01-02,T,0.9
7,John,2018-01-02,T,0.9
8,Sally,2018-01-03,T,0.9
9,Jane,2018-01-03,T,0.9


In [83]:
# group by name and calculate the mean attendance
df = df.groupby(['name'], as_index=False).mean()
df

,name,attendance_value
0,Billy,0.5250
1,Jane,0.6875
2,John,0.9125
3,Sally,0.7625
